In [6]:
import numpy as np
rng = np.random.default_rng()
N = 100
path_hamiltonian = np.zeros((N, N))

#creating a path hamiltonian and random hamiltonian for testing
for i in range(N):
    for j in range(N):
        if (i+1)%N == j or (i-1)%N == j:
            path_hamiltonian[i][j] = -1


A = np.zeros((N, N))
A.fill(.5)
random_hamiltonian = 2*(rng.binomial(1, .5, (N, N)) - A)
random_hamiltonian = (random_hamiltonian + random_hamiltonian.T)/2

for i in range(N):
    random_hamiltonian[i][i] = 0
    
#produces a hamiltonian from sampler data file
def hamiltonian(filename):
    file = pd.read_csv(filename)
    size = len(file.columns)-1
    H = np.zeros((size, size))
    
    for i in range(size):
            for j in range(size-1):
                H[i][j] = file.iat[j, i+1]
            
    return H

#computes the number of indices where two states differ
def discrepancy(state1, state2):
    d1 = sum(abs(state1+state2))/2
    d2 = sum(abs(state1-state2))/2
    return min(d1, d2)

#finds the smallest discrepency between states of minimal energy
def min_discrepancy(min_state, filename):
    row = 0
    sampler_data = pd.read_csv(filename)
    
    sampler_min_energy = sampler_data.at[row, 'energy']
    min_discr = discrepancy(min_state, [sampler_data.iat[row, i+1] for i in range(len(sampler_data.columns)-4)])
    
    while sampler_data.at[row+1, 'energy'] == sampler_min_energy:
        row += 1
        row_discrepancy = discrepancy(min_state, [sampler_data.iat[row, i+1] for i in range(len(sampler_data.columns)-4)])
        if row_discrepancy < min_discr:
            min_discr = row_discrepancy
        
    return min_discr  

In [8]:
import pandas as pd
import time
import ParallelTempering

#time_data = pd.read_csv("Data/Time_Data.csv")
comparison_data = {}
comparison_data_rows = ['PT Min energy', 'DWave Min Energy', 'Discrepancy']
start_time = time.time()

for i in range(1, 11):
    #cliqfile = "Data/Clique_Hamiltonian_" + str(i)
    #pegfile = "Data/Pegasus_Hamiltonian_" + str(i)
    #gaussfile = "Data/Gauss_Hamiltonian_" + str(i)
    cliqgaussfile = "Data/Clique_Gauss_Hamiltonian_" + str(i)
    
    #cliq_hamiltonian = hamiltonian(cliqfile)
    #peg_hamiltonian = hamiltonian(pegfile)
    #gauss_hamiltonian = hamiltonian(gaussfile)
    cliq_gauss_hamiltonian = hamiltonian(cliqgaussfile)
    
    temperatures = np.arange(1, 10, .5)
    
    cliq_time = 0
    peg_time = 0
    
    if i < 6:
        #cliq_time = 3
        #peg_time = 2
        #gauss_time = 2
        cliq_gauss_time = 4
    else:
        #cliq_time = 5
        #peg_time = 60
        #gauss_time = 40
        cliq_gauss_time = 6
        
    
    #cliq_results = ParallelTempering.timed_find_min(cliq_hamiltonian, temperatures, cliq_time)
    #print("Finished Cliq " + str(i) + " at " + str(time.time() - start_time))
    #peg_results = ParallelTempering.timed_find_min(peg_hamiltonian, temperatures, peg_time)
    #print("Finished Peg " + str(i) + " at " + str(time.time() - start_time))
    cliq_gauss_results = ParallelTempering.timed_find_min(cliq_gauss_hamiltonian, temperatures, cliq_gauss_time)
    print("Finished Clique Gauss " +str(i) + " at " + str(time.time()-start_time))
    
    #dwave_cliq = pd.read_csv("Data/Clique_Sampler_Data_" + str(i))
    #dwave_peg = pd.read_csv("Data/Pegasus_Sampler_Data_" + str(i))
    #dwave_gauss = pd.read_csv("Data/Gauss_Sampler_Data_"+str(i))
    dwave_cliq_gauss = pd.read_csv("Data/Clique_Gauss_Sampler_Data_"+str(i))
    
    #comparison_data["Clique "+ str(i)] = [cliq_results[1], 2*dwave_cliq.at[0, 'energy'], min_discrepancy(cliq_results[0], "Data/Clique_Sampler_Data_" + str(i))]
    #comparison_data["Pegasus "+ str(i)] = [peg_results[1], 2*dwave_peg.at[0, 'energy'], min_discrepancy(peg_results[0], "Data/Pegasus_Sampler_Data_" + str(i))]
    comparison_data["Cliq Gauss "+ str(i)] = [cliq_gauss_results[1], 2*dwave_cliq_gauss.at[0, 'energy'], min_discrepancy(cliq_gauss_results[0], "Data/Clique_Gauss_Sampler_Data_"+str(i))]
    

Finished Clique Gauss 1 at 4.218276023864746
Finished Clique Gauss 2 at 8.34195590019226
Finished Clique Gauss 3 at 12.445865869522095
Finished Clique Gauss 4 at 16.521746158599854
Finished Clique Gauss 5 at 20.594604015350342
Finished Clique Gauss 6 at 27.04023003578186
Finished Clique Gauss 7 at 33.51032304763794
Finished Clique Gauss 8 at 39.93681502342224
Finished Clique Gauss 9 at 46.404550075531006
Finished Clique Gauss 10 at 52.894739866256714


In [21]:
#data = pd.DataFrame(comparison_data, index = comparison_data_rows)
#data.to_csv("Data/Comparison_data.csv")

In [25]:
import matplotlib.pyplot as plt


In [21]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Pegasus 1,Pegasus 2,Pegasus 3,Pegasus 4,Pegasus 5,Pegasus 6,Pegasus 7,...,Cliq Gauss 1,Cliq Gauss 2,Cliq Gauss 3,Cliq Gauss 4,Cliq Gauss 5,Cliq Gauss 6,Cliq Gauss 7,Cliq Gauss 8,Cliq Gauss 9,Cliq Gauss 10
0,0,0,PT Min energy,-167.0,-151.0,-147.0,-161.0,-155.0,-2262.0,-2302.0,...,-348.480086,-339.648294,-331.305390,-369.551304,-320.889610,-1898.711254,-1870.459331,-1856.772057,-1876.264280,-1881.253970
1,1,1,DWave Min Energy,-168.0,-156.0,-148.0,-164.0,-160.0,-2208.0,-2180.0,...,-356.840903,-345.971195,-337.964804,-372.267325,-327.342844,-1733.736907,-1718.437293,-1740.065592,-1729.620283,-1731.617204
2,2,2,Discrepancy,0.0,0.0,0.0,0.0,0.0,217.0,165.0,...,1.000000,0.000000,17.000000,0.000000,0.000000,39.000000,32.000000,59.000000,45.000000,56.000000


In [33]:
plt.plot(['Pegasus Subgraph, size 40'], 1, '-b' ['Pegasus Subgraph, size 448'], 2, '-b')

<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-33-5b0228de46e2>:1: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  plt.plot(['Pegasus Subgraph, size 40'], 1, '-b' ['Pegasus Subgraph, size 448'], 2, '-b')


TypeError: string indices must be integers